# Tutorial 05 - Calibración de cámara

Agenda:

- Checkerboard
- Modelos de Cámara 
  - distancia focal
  - pinhole
  - modelo de camara con distorsión 
- Calibración Monocular
- Rectificación
- Estimación de pose


# Setup

Este tutorial se puede ejecutar local en Jupyter lab o utilizar Google Colab.

## En Google Colab 
Este tutorial se provee junto con archivos de recursos dentro de un archivo ".zip".
En caso de ejecutar en Google Colab hay que:

1. Descomprimir el zip en algún lado
2. Subir el contenido del zip a Google Drive en alguna carpeta (por ejemplo `udesa/I308/tutoriales/tutorial_X`)
3. Abrir este notebook .ipynb 

In [ ]:
import os
import sys

# TODO: establecer el path en caso de trabajar con Colab
DRIVE_DIR = "udesa/I308/tutoriales/tutorial_05"

# detecta si estamos corriendo en Google Colab
try:
  from google.colab import drive
  COLAB = True
except:
  COLAB = False

if COLAB:
    # monta Google Drive
    drive.mount('/content/drive')

    base_path = "/content/drive/MyDrive/"
    path = os.path.join(base_path, DRIVE_DIR)
    
    %cd {path}
    sys.path.append(path)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# instalamos el paquete de utilidades
!pip install -qq git+https://github.com/udesa-vision/i308-utils.git

from i308_utils import imshow, show_images

In [ ]:
# descarga dataset de calibración monocular
from setup import download_calib_dataset

download_calib_dataset()

# Modelos De Cámara

## Para qué queremos tener un modelo de cámara?

- Si tenemos un modelo matemático de la cámara, podemos describir
la relación que existe entre las coordenadas de un punto en 3d en el mundo 
y su proyección en el plano de la imagen.

- Calibrar una cámara es fundamental para obtener mediciones precisas y realizar correcciones en las imágenes capturadas, por ejemplo corregir los errores introducidos por distorsiones de la lente.

- El modelo de cámara nos permite conocer la relación exacta entre las coordenadas 3D del mundo real y sus proyecciones 2D en la imagen.

- Aplicaciones 3D

  - Realidad aumentada: podemos colocar objetos virtuales en imágenes en posiciones y orientaciones precisas.
  - Reconstrucción 3D: podremos reconstruir escenas o objetos en 3D a partir de imágenes.
  - Medir distancias y tamaños de manera precisa
  - Tracking de objetos.
  - [VPS](https://geospatialworld.net/prime/business-and-industry-trends/what-is-visual-positioning-system-vps/)

## Qué modelos de cámara vamos a usar hoy?
- modelo sólo usando distancia focal
- modelo pinhole
- modelo de camara con distorsión


Los dos primeros modelos vamos a calibrarlos "a mano", con código propio.
Para entender de manera didáctica cómo podría resolverse.
Para el último modelo utilizaremos OpenCV.



# Checkerboard

¿Qué es un Checkerboard?

Un checkerboard es un patrón de calibración que tiene características que facilitan su detección mediante computer vision.

- Se compone por casillas en forma de mosaico, y va alternando blanco y negro como un tablero de ajedrez. 

- Este tipo de patrones podemos armarlos muy facilmente mediante OpenCV de manera sintética, o bien utilizar un [sitio web como este](https://markhedleyjones.com/projects/calibration-checkerboard-collection) para generarlo.
De esa manera si queremos podemos controlar el tamaño de cada celda.

- Para trabajar con este tipo de patrones los algoritmos van a asumir que todos los corners están en un plano, por lo que es conveniente pegar la hoja sobre una superficie plana y rígida.

- Para evitar ambigüedades es conveniente que utilicemos una cantidad par por una cantidad impar de celdas, o vice-versa.


In [ ]:
import cv2
board_image = cv2.imread("res/checkerboard.jpg", 0)
imshow(
    board_image,
    title="checkerboard 6x8"
)


## Detectando un checkerboard

para encontrar los corners del checkerboard podemos usar la función de OpenCV [cv2.findChessboardCorners](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a)

Esta función recibe:
- la imagen donde buscar (en grayscale)
- el checkerboard (un par 6x8 en este caso)

y devuelve el par: (found, corners)

- found: True o False, si lo encontró o no
- corners: un array con las posiciones de los corners encontrados


In [ ]:
# Find the chess board corners

checkerboard = (6, 8)

found, corners = cv2.findChessboardCorners(
    board_image,
    checkerboard,
    cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE
)



In [ ]:
found

In [ ]:
# Obs: los corners retornados vienen en un array de (Nx1x2),
# por eso muchas veces es necesario reshapearlos a (Nx2)
corners.shape

In [ ]:
corners.reshape(-1, 2)[:3]

## Mejorando precisión de los corners encontrados

Si encontramos el checkerboard, entonces OpenCV nos da la funcion [cv2.cornerSubPix](https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e), donde implementa un algoritmo iterativo para mejorar la localización de los corners encontrados a nivel sub-pixel

In [ ]:
# Encontramos los corners
# refinemos las coordenadas encontradas
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# refining pixel coordinates for given 2d points.
corners = cv2.cornerSubPix(
    board_image, 
    corners, 
    (11, 11), 
    (-1, -1), 
    criteria
)



En general vamos a querer ejecutar ambas funciones en secuencia: 

1. cv2.findChessboardCorners
2. cv2.cornerSubPix

Para simplificar nuestro código podemos armar una función que realice ambas invocaciones.
O bien usar la función `detect_board` que se provee en el modulo `calib.py`

## Graficando un checkerboard

1. OpenCV provee la función [cv2.drawChessboardCorners](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga6a10b0bb120c4907e5eabbcd22319022) para graficar los corners detectados.

2. También se provee la función `draw_checkerboard` dentro del modulo `calib.py`, que hace una tarea similar,
pero permite controlar el thickness y tamaño los circulos en el dibujo (útil para imagenes con mucha resolución)
ya que aparentemente con drawChessboardCorners no es posible tocar esos parámetros.

Ejemplo:
```python

import calib

found, corners = calib.detect_board(checkerboard, image)

````


In [ ]:
import calib

show_board = cv2.cvtColor(board_image, cv2.COLOR_GRAY2BGR)

# dibujo los corners usando la función drawChessboardCorners de OpenCV
show_board_cv = cv2.drawChessboardCorners(
    show_board.copy(), 
    checkerboard, 
    corners, 
    found
)

# dibujo los corners usando la función drawChessboardCorners de OpenCV
show_board_mine = calib.draw_checkerboard(
    show_board.copy(),
    checkerboard,
    corners,
    found
)

show_images([
    #board_image,
    show_board_cv,
    show_board_mine
], ["drawChessboardCorners (CV)", "draw_checkerboard (Custom)"])

Observemos que como OpenCV sabe el layout del checkerboard,
los corners devueltos son unívocos y están ordenados.

Entonces puedo acceder por índice al array resultante, y se exactamente cuál es 
ese corner.

Por ejemplo que los boundings de los corners estarán en los indices:
- $0$,
- $checkerboard_w - 1$,
- $checkerboard_h * (checkerboard_h - 1)$ y en
- $(checkerboard_w * checkerboard_h) - 1$ 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

ch_w, ch_h = checkerboard

pts = corners.reshape(-1, 2)

boundings = np.array([
    pts[0],
    pts[ch_w - 1],
    pts[(ch_w * ch_h) - 1],
    pts[(ch_w * (ch_h - 1))],
])
_, ax = imshow(board_image, show=False)
ax.scatter(boundings[:, 0], boundings[:, 1], marker='x', color='r')
plt.show()


# Modelo de cámara "Distancia Focal"

En su versión más simple, nuestro modelo de cámara es **un número: la distancia focal**.


Si asumimos que:
- nuestra cámara se comporta como pinhole, no hay distorsiones causadas por lentes.
- nos mantenemos siempre centrados al checkerboard de manera perpendicular (en la direccion del eje óptico)

=> podemos usar similaridad de triángulos para estimar f.

Cómo?



## Estimando Distancia Focal

In [ ]:
# 1. Medimos nuestro checkerboard para verificar el tamaño de cada casilla
# En este caso el tamaño de cada casilla es de 24.2 mm

square_size_mm = 24.2
# ancho en casillas x alto en casillas
checkerboard = (10, 7)




<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/measure.jpg?raw=true" width="80%"/>

OBS: como todas las casillas son del mismo tamaño, podemos medir varias y dividir por la cantidad, y obtendremos menos error de medición.

In [ ]:
# 2. usamos una cinta métrica y nos alejamos una distancia conocida, para capturar una imagen del checkerboard.
img = cv2.imread("res/images_measured/dist_100cm.jpg", 0)
distance_mm = 1000

# img = cv2.imread("recursos/images_measured/dist_200cm.jpg", 0)
# distance_mm = 2000

# img = cv2.imread("recursos/images_measured/dist_40cm.jpg", 0)
# distance_mm = 400



distance_m = "{:.2f}".format(distance_mm / 1000)
imshow(img, title=f'distancia conocida={distance_m}m', figsize=(10, 6))


In [ ]:
# 3. Ahora tomemos imagenes a distancia desconocida:
# En realidad cada archivo tiene encodada la distancia a la que fue tomada la foto
# pero supongamos que estamos en "runtime" donde no conocemos la distancia.


img1 = cv2.imread("res/images_measured/dist_40cm.jpg", 0)
img2 = cv2.imread("res/images_measured/dist_70cm.jpg", 0)
img3 = cv2.imread("res/images_measured/dist_100cm.jpg", 0)
img4 = cv2.imread("res/images_measured/dist_150cm.jpg", 0)
img5 = cv2.imread("res/images_measured/dist_200cm.jpg", 0)


show_images([
    img,
    img1,
    img4
], [
    f"checkerboard a {distance_m}m",
    "checkerboard a ?",
    "checkerboard a ?",
])

In [ ]:
# Cómo podemos estimar la distancia de la cámara a los checkerboards?

In [ ]:
import misc

misc.plot_simplest_camera_model(
    obj_x = distance_mm,
    objs_x_unk=[400]
)

Por similitud de triángulos tenemos:

$\displaystyle \frac{f}{p} = \frac{d}{w} $ 

En donde:
- $ w $ [mm] es el ancho del objeto en el mundo
- $ p $ [mm] es la distancia al objeto
- $ p $ [px] es el ancho del objeto en píxeles

Entonces:

$\displaystyle f = \frac{d \times p}{w} $

In [ ]:
# 4. detectamos el checkerboard en la imagen conocida
#  se provee la función detect_board que realiza ese trabajo en el modulo calib
import calib

checkerboard = (10, 7)
found, corners = calib.detect_board(checkerboard, img)

show_board = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

show_board = calib.draw_checkerboard(
    show_board,
    checkerboard,
    corners,
    found
)
imshow(show_board, figsize=(10, 6))

In [ ]:
# 5. tomamos solamente los corners c1=(0,0) y c2=(0, 9). c3 = (7, 9)

pts = corners.reshape(-1, 2)

c1 = pts[0]
c2 = pts[9]
c3 = pts[10 * 7 - 1]

_, ax = imshow(show_board, figsize=(10, 6), show=False)
ax.scatter(c1[0], c1[1], marker='x', color='r')
ax.scatter(c2[0], c2[1], marker='x', color='r')
ax.scatter(c3[0], c3[1], marker='x', color='r')

plt.show()

In [ ]:
# 6. tenemos las posiciones en pixeles, podemos obtener el ancho (y el alto) en píxeles del checkerboard

w_px = np.linalg.norm(
    c1 - c2
)
h_px = np.linalg.norm(
    c3 - c2
)
# w_px = np.abs(c2[0] - c1[0]) 

w_px, h_px

In [ ]:
# 7. conocemos el tamaño del checkerboard en el mundo real (ancho en x)
# calculémoslo:

square_size_mm = 24.2
object_size_mm = (np.array(checkerboard) - (1, 1)) * square_size_mm
object_size_mm

In [ ]:
w_px

In [ ]:
# 8. Listo! tenemos: 
# - la distancia en profundidad, 
# - el tamaño en pixeles
# - el tamaño en el mundo real, 
# => podemos calcular la distancia focal.

# w_px / f = w_world / distance
# => f = w_px * distance / w_world

# fx: focal len x
obj_w_mm, obj_h_mm = object_size_mm

fx = distance_mm * w_px / obj_w_mm
fy = distance_mm * h_px / obj_h_mm


In [ ]:
print("distance camera [mm]:", distance_mm)
print("object world size [mm]:", np.round(obj_w_mm, 1), np.round(obj_h_mm, 1))
print("object image size [px]:", np.round(w_px), np.round(h_px))
print("focal lenght (fx, fy):", np.round(fx, 1), np.round(fy, 1))


## Estimando distancia al objeto usando la distancia focal

In [ ]:
# Ahora que tenemos nuestro modelo de cámara:
#
# Creemos una función que dados:
#
#     - el tamaño del checkerboard
#     - una imagen del checkerboard 
#     - la distancia focal
#     - el tamaño del checkerboard en el mundo
#     devuelve a qué distancia está el checkerboard.

import calib

def estimate_distance(
    checkerboard, 
    img, 
    fx,
    obj_width_mm,
):

    # Encuentra el checkerboard
    found, corners = calib.detect_board(checkerboard, img)
    if not found:
        raise Exception("board not found")

    pts = corners.reshape(-1, 2)
    ch_w, ch_h = checkerboard
    c1 = pts[0]
    c2 = pts[ch_w - 1]
    c3 = pts[ch_w * ch_h - 1]

    w_px = np.linalg.norm(
        c1 - c2
    )

    #h_px = np.linalg.norm(
    #    c3 - c2
    #)

    # Estima la distancia
    distance_mm = obj_width_mm * fx / w_px

    return distance_mm


    

In [ ]:
images = [
        img1,
        img2,
        img3,
        img4,
        img5
    ]


estimated_distances = [
    # para cada imagen, estimamos la distancia
    estimate_distance(
        checkerboard,
        img,
        fx,
        obj_width_mm=obj_w_mm,
    )
    for img in images

]


In [ ]:
real_distances = [
    400,
    700,
    1000,
    1500,
    2000
]

for img, d_hat, d in zip(
    images,
    estimated_distances, 
    real_distances
):
    err = 100 * np.abs(d - d_hat) / d
    title = "real[mm]: {:.1f}, estim[mm]: {:.1f}, err rel: %{:.1f}".format(
        d, 
        d_hat,
        err
    )
    imshow(img, title=title, figsize=(10, 6))



# Modelo de cámara Pinhole 

Repasemos,

<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/pinhole_setup.png?raw=true" width="100%"/>

$X_w$ y $X_c$ es mismo punto, pero visto desde distintos marcos de referencia.

- $ X_w = (x_w, y_w, z_w)$ (en el marco de referencia del mundo).
- $ X_c = (x_c, y_c, z_c)$ (en el marco de referencia de la cámara).
- $ p = (u, v)$ (las coordenadas del pixel donde cae este punto en la imagen).

El marco de referencia de la cámara está centrado en el pinhole y con z alineado con el eje óptico,
que en la imagen pasa por el punto principal.



## Parámetros Intrínsecos

Los parámetros intrínsecos de la cámara son básicamente la distancia focal y el punto principal: $f_x, f_y, o_x, o_y $.

Tenemos "dos distancias focales" $f_x$, $f_y$, para compensar pixeles no cuadrados, dadas $m_x$ y $m_y$, las densidades de píxeles por milímetro en cada eje, $f_x = f . m_x$, $f_y = f . m_y$. 

### Matriz de Calibración K

Disponemos los parámetros intrínsecos de la cámara para formar la homografía $K$:

$K = \begin{bmatrix}
    f_x & 0 & o_x  \\
    0 & f_y & o_y  \\
    0 & 0 & 1  \\
\end{bmatrix}
$

Obs: $K$ es triangular superior.

### Matriz Intrínseca Mint

Es una matriz $M_{int}$ en $(3 \times 4)$ que se arma concatenando K con un vector de ceros:

$M_{int} = \begin{bmatrix}
    f_x & 0 & o_x & 0 \\
    0 & f_y & o_y & 0 \\
    0 & 0 & 1 & 0 \\
\end{bmatrix}
= \begin{bmatrix}
    K | 0 
\end{bmatrix}
$

Esta matriz toma un punto 3d un punto en el marco de referencia de la cámara, expresado en coords homogeneas, y lo lleva de a un píxel en la imagen en coordenadas homogéneas.

$ \widetilde{p} = M_{int} . \widetilde{X_c} $

## Parámetros Extrínsecos

Los parámetros extrínsecos son:

- la traslación $t$, y
- la rotación $R$

de la cámara con respecto al marco de referencia del mundo.

$R = \begin{bmatrix}
    r_{11} & r_{12} & r_{13} \\
    r_{21} & r_{22} & r_{23} \\
    r_{31} & r_{32} & r_{33} \\
\end{bmatrix}
;
t = \begin{bmatrix}
    t_{x} \\
    t_{y} \\
    t_{z} \\
\end{bmatrix}
$

### Rotación
- $R$ es una matriz de Ortonormal.
- Cada fila de $R$ representa la dirección de los versores de la cámara en el marco de referencia del mundo.

### Traslación
Este es un vector de tres elementos que indica la distancia de la cámara en las direcciones x, y, y z del sistema de coordenadas del mundo.

- $ t = -R . c_w $

### Matriz extrínseca Mext

$M_{ext}$ en $(4 \times 4)$ que se arma con los parámetros extrínsecos:

$M_{ext} = \begin{bmatrix}
    r_{11} & r_{12} & r_{13} & t_x\\
    r_{21} & r_{22} & r_{23} & t_y\\
    r_{31} & r_{32} & r_{33} & t_z \\
    0 & 0 & 0 & 1 \\
\end{bmatrix}
= \begin{bmatrix}
    R & t \\
    0 & 1 \\
\end{bmatrix}
$

Esta matriz toma un punto 3d en el marco de referencia del mundo, expresado en coords homogeneas, y lo lleva a un punto en 3d en el marco de referencia de la cámara.

$ \widetilde{X_c} = M_{ext} . \widetilde{X_w} $

## Matriz de Proyección P

La matriz de Proyección $ P $, que se puede armar componiendo las matrices intrínseca y extrínseca.

$P = M_{int} . M_{ext} $

$
P = \begin{bmatrix}
    p_{11} & p_{12} & p_{13} & p_{14} \\
    p_{21} & p_{22} & p_{23} & p_{24} \\
    p_{31} & p_{32} & p_{33} & p_{34} \\
\end{bmatrix}
$

Esta matriz $P$ lleva un puntos desde las coordenadas del mundo 3d a píxeles en la imagen 2d.

$ \widetilde{p} = P . \widetilde{X_w} $



## Estimando modelo Pinhole con un objeto 3D conocido

Necesitamos un objeto 3d con medidas conocidas.

Vamos a calibrar la cámara usando la mesita de luz:

<div style="display: flex;">
  <img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/night_table/measuring_night_table.jpg?raw=true" alt="midiento mesita de luz" style="width: 50%; margin-right: 10px; object-fit: cover;">
  <img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/night_table/measuring_night_table_model.jpeg?raw=true" alt="plano 3D" style="width: 50%; object-fit: cover;">
</div>

1. Usando una cinta métrica, mido las dimesiones de la mesita de luz.
2. Con eso puedo armar un plano 3d de la mesita
3. Si asumo que el $(0, 0)$ es el vértice superior izquierdo de atrás, puedo saber donde está en 3d cada vértice (en coordenadas del mundo).


In [ ]:
# coloco los puntos conocidos en 3d del objeto en un array.

obj_points = np.array([
    
    # (0, 0, 0),
    (45, 0, 0),
    (45, 0, -40),
    # (45, 57.5, -40),
    (0, 57.5, -40),
    (0, 57.5, 0),
    (0, 0, -40),

    # (45, 23.75, -40),
    (15, 23.75 + 7, -40),
    (45 - 15, 23.75 + 7, -40),

    # (22.5, 23.75, -40 + 3),
    (0, 23.75, -40 + 3),
    
])

# usando GIMP busco las correspondencias a mano

img_points = np.array([
    # (701, 182),
    (1163, 76),
    (1507, 181),
    # (1424, 805),
    (996, 1017),
    (795, 746),
    (942, 346),

    # (1468, 478),
    (1148, 685),
    (1314, 617),

    # (1213, 546),
    (948, 651),
    
])

In [ ]:
img = cv2.imread("res/night_table/night_table_1.jpg", 0)

_, ax = imshow(img, show=False, figsize=(10, 6))
ax.scatter(img_points[:, 0], img_points[:, 1], marker='+', color='r')
plt.show()

Usando pares de correspondencias, armo un sistema de ecuaciones, que expreso de manera matricial.
Muy parecido a lo que hicimos antes para estimar la H.
La diferencia ahora es que $P$ es de $(3 \times 4)$.

$ \widetilde{p} = P . \widetilde{X_w} $

- P tiene DoF=11.
- Cada correspondencia aporta 2 ecuaciones.
- Cuántos pares de correspondencias necesitamos?

Esta vez resolvemos usando cuadrados mínimos, para eso aplicamos SVD y luego tomamos el autovector correspondiente al menor autovalor.

<span style="color:red">OJO: este método **NO FUNCIONA** si los puntos en el mundo son coplanares.</span> 


In [ ]:
def estimate_projection_matrix(world_points, image_points):
    """
        Estimamos la matrix de proyección P usando pares de correspondencias
        entre puntos en el mundo y puntos en la imagen.
    """
    
    # Construye el sistema de ecuaciones de forma matricial,
    # como la matriz A
    A = []
    for i in range(len(world_points)):
        x_w, y_w, z_w = world_points[i]
        u_i, v_i = image_points[i]
        
        A.append([x_w, y_w, z_w, 1,   0,   0,   0, 0, -u_i * x_w, -u_i * y_w, -u_i * z_w, -u_i ])
        A.append([  0,   0,   0, 0, x_w, y_w, z_w, 1, -v_i * x_w, -v_i * y_w, -v_i * z_w, -v_i ])

    A = np.array(A)
    
    # Usamos SVD para resolver el sistema
    _, _, Vt = np.linalg.svd(A)
    # el vector p que se corresponde con el menor auto-valor de A, 
    # es en efecto la matriz P, dispuesta como vector.
    p = Vt[-1, :]  

    # Reshape p to form the projection matrix P
    P = p.reshape(3, 4)

    # Normalizo P
    P /= P[2, 3] 

    return P


In [ ]:
P = estimate_projection_matrix(obj_points, img_points)

In [ ]:
P.round()

In [ ]:
# Usando la matriz P puedo proyectar puntos del mundo en la imagen:

# cara lateral izquierda de la mesita
face1 = np.array([
    (0,  0, 0),
    (0, 0, -40),
    (0, 57.5, -40),
    (0,  57.5, 0),
])

# borde superior lateral derecho de la mesita
face2 = np.array([
    (45,  0,   0),
    (45,  0, -40),
    (45,  4, -40),
    (45,  4,   0),
])

# borde inferior lateral derecho de la mesita
face3 = np.array([
    (45,  23.75,   0),
    (45,  23.75, -40),
    (45,  57.5,  -40),
    (45,  57.7,    0),
])


In [ ]:
def project_points(P, world_points):
    # Convert world points to homogeneous coordinates (Nx4)
    wph = np.hstack((world_points, np.ones((world_points.shape[0], 1))))
    
    # Project world points onto the image plane
    proj_h = wph @ P.T
    
    # Convert to 2D image coordinates by normalizing with the third homogeneous coordinate
    proj = proj_h[:, :2] / proj_h[:, 2].reshape(-1, 1)

    return proj

In [ ]:
from matplotlib.patches import Polygon
def draw_face(ax, face, color):
    ax.scatter(face[:, 0], face[:, 1], marker='+', color='g')
    ax.add_patch(Polygon(face, alpha=0.5, color=color))

In [ ]:

f1_p = project_points(P, face1)
f2_p = project_points(P, face2)
f3_p = project_points(P, face3)

_, ax = imshow(img, figsize=(10, 6), show=False)
draw_face(ax, f1_p, color='y')
draw_face(ax, f2_p, color='m')
draw_face(ax, f3_p, color='c')
plt.show()


In [ ]:
# Obs: incluimos puntos que están ocluidos, pero aún así sabemos donde se proyectan.


## Descomposición de P

Podemos descomponer P = $M_{int}$ . $M_{ext}$, usando la factorización QR.

- El centro de camara está en el null space de P. $ P . c = 0 $
Podemos encontrar $c$ usando SVD(P) y luego tomando el auto-vector que se corresponde con el menor autovalor.

- Luego usamos que:

$
P = \begin{bmatrix}
    p_{11} & p_{12} & p_{13} & p_{14} \\
    p_{21} & p_{22} & p_{23} & p_{24} \\
    p_{31} & p_{32} & p_{33} & p_{34} \\
\end{bmatrix}
= 
\begin{bmatrix}
    K & 0 \\
\end{bmatrix}
 \begin{bmatrix}
    R & t \\
    0 & 1 \\
\end{bmatrix}
$

Luego:

$ 
K \begin{bmatrix}
    R | t \\
\end{bmatrix}
= K  \begin{bmatrix}
    R | -R . c \\
\end{bmatrix}
=  \begin{bmatrix}
    M | -M . c \\
\end{bmatrix}
$

Aplicamos despomosición RQ en M

$ M = K . R $


In [ ]:
def factorize_projection_matrix(P):
    
    # Nos aseguramos de que P sea una matrix (deberia ser de 3x4)
    P = np.asarray(P)
    
    # Extraemos la parte superior izquerda de P (3x3) que se corresponde 
    # con K * R
    M = P[:, :3]
    
    # Computamos el centro de cámara 
    # usando el null space de P
    U, S, Vt = np.linalg.svd(P)
    C = Vt[-1, :]
    C = C / C[-1]  # normalizamos el vector homogeneo
    
    # Realizamos la descomposición RQ de M 
    # con eso obtenemos R y K
    R, K = np.linalg.qr(np.linalg.inv(M))
    
    # Invertimos para obtener la K y R correctas 
    # de la factorización RQ.
    K = np.linalg.inv(K)
    R = np.linalg.inv(R)
    
    # Se asegura de que K tiene elementos positivos en la diagonal.
    T = np.diag(np.sign(np.diag(K)))
    K = np.dot(K, T)
    R = np.dot(T, R)
    
    # Normaliza K, dividiendo por K[2, 2]
    K = K / K[2, 2]
    
    # Vector de traslación t
    # (P[:, 3] es t que es la última columna de P)
    t = np.dot(np.linalg.inv(K), P[:, 3])
    
    return K, R, t

In [ ]:
K, R, t = factorize_projection_matrix(P)

In [ ]:
# la distancia focal de K nos dio parecida a la calibración anterior :)
K.astype(int)

In [ ]:
# Chequeemos que R es una matriz Ortonormal :)
R @ R.T

In [ ]:
np.linalg.norm(R, axis=1)

## Limitaciones del modelo Pinhole.

Pinhole no modela efectos como:

- Distorsión de la lente
- Aberraciones ópticas
- Vignetting
- Profundidad de campo y desenfoque
- Efectos de dispersión y reflexión


# Modelo de cámara de OpenCV ~ Pinhole + Distorsión

El modelo de cámara que emplea OpenCV, es una extensión del modelo pinhole, donde se incluye el modelado de la distorsión causada por la lente.

## Calibrando una cámara con OpenCV

Podemos calibrar la cámara usando la función de OpenCV [`cv2.calibrateCamera`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d).

Esta función básicamente recibe un conjunto de correspondencias mundo-imagen $3d \leftrightarrow 2d$, y nos devuelve los parámetros cámara: (la matriz intrínseca `K` y los coeficientes de distorsion `dist_coeffs`).



Ejemplo de uso:

```python
    ret, K, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        world_points,
        image_points,
        img_shape, None, None
    )
```

**Argumentos**

- **image_points**: es una lista de puntos detectados en la imagen. Cada elemento de esta lista será un numpy array con puntos en 2d.

- **world_points**: es una lista de puntos en el mundo. Cada elemento de esta lista será un numpy array con puntos en 3d.

- **img_shape**: es el shape de las imagenes de entrada.
    

**Resultado**

`cv2.calibrateCamera` devuelve tanto los parámetros intrínsecos y extrínsecos de la cámara como los coeficientes de distorsión, proporcionando todo lo necesario para corregir la distorsión en imágenes futuras y entender cómo la cámara se posicionó en el espacio durante la captura de las imágenes de calibración.


- **ret**: Es el error promedio de reproyección. Este valor representa la precisión de la calibración. Cuanto menor sea el valor, mejor será la calibración. En esencia, indica la cantidad promedio de error (en píxeles) entre los puntos proyectados y los puntos detectados en las imágenes.
  
- **K**: la matriz de calibración

$K = \begin{bmatrix}
    f_x & 0 & o_x  \\
    0 & f_y & o_y  \\
    0 & 0 & 1  \\
\end{bmatrix}
$

- **dist_coeffs**: Es un vector que contiene los coeficientes de distorsión de la lente. Estos coeficientes describen las distorsiones ópticas de la cámara, como la distorsión radial y tangencial. Para el modelo estándar de distorsión en OpenCV, este vector tiene la forma $[k_1, k_2, p_1, p_2, k_3]$, en donde:

  - $k_1, k_2, k_3$ son los coeficientes de distorsión radial
  - $p_1, p_2$ son los coeficientes de distorsión tangencial

- **rvecs**: Es una lista de vectores de rotación. Cada vector de rotación representa la rotación de la cámara con respecto al sistema de coordenadas del mundo para cada imagen utilizada en la calibración. Estos vectores pueden ser convertidos a matrices de rotación si es necesario.

- **tvecs**: Es una lista de vectores de traslación. Cada vector de traslación representa la posición de la cámara en el espacio 3D del mundo con respecto al sistema de coordenadas del mundo para cada imagen utilizada en la calibración.


### OJO - Deshabilitar Auto Foco!

Muchas cámaras digitales, pueden tener Auto-Foco, que es básicamente un motor para ajustar la posición de la lente y así enfocar correctamente el objeto de la imagen. Si la lente se mueve, cambia la distancia focal y el punto principal. Con lo cual **CON AUTOFOCO NO NOS SIRVE LA CALIBRACIÓN REALIZADA**.

Otras cámaras tienen la lente fija, y por lo tanto los parámetros intrínsecos son fijos.

#### En Android
Para deshabilitar el autofoco en un smartphone Android podemos configurarlo en modo "pro".
En la app de cámara hay un deslizador en la parte de abajo, que moviéndolo hacia la derecha, puede verse la opción "Pro". OBS: No todo smartphone cuenta con esta opción.

  <img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/camera_pro_mode.jpeg?raw=true" alt="Android Pro Mode" width="30%">

En modo Pro. Seleccionamos el botón "MF" para evitar el foco automático.
Deberíamos ver otro un slider, que debemos llevar al infinito.

La calibración servirá mientras no toquemos este setting.
  

#### En cámara LogiTech

Para el caso de la cámara logitech, se puede utilizar el [software logitune](https://www.logitech.com/en-us/video-collaboration/software/logi-tune-software.html?srsltid=AfmBOopFp5qn45SXHpnh6D6kDtT8dglM0V2taHPdgyJDcvh7Y16ziK4d) que permite desactivar el autofoco de la cámara

  <img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/logi_tune.png?raw=true" alt="Logi Tune" width="30%">


## Procedimiento de calibración

OpenCV utiliza para calibrar el método de [Zhang](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr98-71.pdf).


1. Preparación: Utilizamos un checkerboard, de medidas conocidas. 

2. Capturamos Imágenes: Se toman múltiples fotos del patrón desde diferentes ángulos y distancias con la cámara que se quiere calibrar (minimo 10, idealmente 20 o más). Es importante que el patrón cubra diferentes partes del campo de visión de la cámara en cada imagen.

3. Detectamos los corners de cada imagen. Mapeamos cada corner a un punto conocido del patrón. Para esto podemos usar las funciones de OpenCV **`cv2.findChessboardCorners`** y **`cv2.cornerSubPix`**.

4. Usamos la función de OpenCV [**`cv2.calibrateCamera`**](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d) que calcula los parámetros intrínsecos de la cámara (como la distancia focal y el punto principal) y los parámetros de distorsión (que corrigen las imperfecciones del lente). También se calculan los parámetros extrínsecos (posición y orientación de la cámara respecto al patrón) para cada imagen.

OpenCV optimizará los parámetros calculados para minimizar el error entre los puntos proyectados según el modelo de cámara y los puntos detectados en las imágenes.

<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/zhang_calibration_procedure.jpeg?raw=true" width="100%"/>


# Calibración en Notebook

Utilizando OpenCV, calibrar la cámara monocular con la que se generó el dataset de calibración del directorio `images`.

El checkerboard utilizado es de: 10x7, con 24.2 mm de casilla


In [ ]:
# Configuro el checkerboard
checkerboard = (10, 7)
square_size_mm = 24.2


In [ ]:
# Cargo las imágenes:
import glob

directory = "./images/*.jpg"
rotation = None  # si las imagenes deben ser rotadas, defino rotation, si no dejo None
scale = 1.0

file_names = glob.glob(directory)

# listo archivos en el directorio
image_files = sorted(
    glob.glob(directory),
    key=lambda f: int(f.split("_")[-1].split(".")[0])
)


In [ ]:
image_files[:3]

In [ ]:
import cv2

# leo las imagenes
images = [
    # Obs: si la imagen se saca con un celular, es posible
    # que al leerla OpenCV nos la rote automáticamente
    # dependiendo del sensor de orientación del celular al momento de tomar la foto
    # para evitar eso podemos usar el flag cv2.IMREAD_IGNORE_ORIENTATION
    cv2.imread(file, 0|cv2.IMREAD_IGNORE_ORIENTATION)
    for file in image_files
]

In [ ]:
len(images)

In [ ]:
if rotation is not None:
    print("rotating images...")
    images = [
        cv2.rotate(img, rotation)
        for img in images
    ]

In [ ]:
test_indices = [5, 10, 15, 20]

In [ ]:
# mostremos algunas de las imágenes del dataset:
show_images([
    images[i] for i in test_indices
], grid=(2, 2), figsize=(10, 6))

In [ ]:
import calib

# armamos un modelo del mundo con los corners del tablero
object_points = square_size_mm * calib.board_points(checkerboard)

# miremos que pinta tienen los 3 primeros puntos:
object_points[:, :3]
#object_points

In [ ]:
image_shape = None
good_images = []
image_points = []  # aquí acumularemos corners detectados en imagenes
world_points = []  # aquí acumularemos los correspondientes puntos en el mundo

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# iteramos, para cada archivo de imagen:
for file, image in zip(image_files, images):

    print(f"detectando {file}...")
    
    shape = image.shape[::-1]
    if image_shape is None:
        image_shape = shape
    else:
        # validemos que todas las imagenes sean del mismo tamaño
        if image_shape != shape:
            raise Exception(f"se incluyeron imágenes de distintos tamaños? {shape} != {image_shape}")

    # detectamos el checkerboard
    # print("finding checkerboard")
    w, h = image.shape[1], image.shape[0]
    scaled = cv2.resize(image, (int(w * scale), int(h * scale)))
    
    found, corners = cv2.findChessboardCorners(
        scaled,
        checkerboard,
        cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE
    )

    if not found:
        print("failed")
    else:

        corners /= scale
        # lo encontramos.

        # intentamos refinar las coordenadas a precisión de sub-pixel
        # print("subpix")
        corners = cv2.cornerSubPix(image, corners, (11, 11), (-1, -1), criteria)

        # acumulamos esta imagen y sus correspondencias
        world_points.append(object_points)
        image_points.append(corners)
        good_images.append(image)


In [ ]:
print(f"se encontraron {len(good_images)} checkerboards / {len(image_files)}")

images = good_images


In [ ]:
# grafiquemos algunos checkerboards para verificar que lo que encontró está ok

show_images([
    calib.draw_checkerboard(
        cv2.cvtColor(images[i], cv2.COLOR_GRAY2BGR),
        checkerboard,
        image_points[i],
        True,
        # corner_radius=10,
        # corner_thickness=5,
        # line_thickness=5
    ) for i in test_indices
], grid=(2, 2), figsize=(10, 6))


In [ ]:
# Calibramos:

ret, K, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    world_points,
    image_points,
    image_shape, None, None
)


In [ ]:
K.astype(int)

In [ ]:
dist_coeffs

In [ ]:
from i308_utils import np_print

# La calibración fue realizada, 
# los parámetros de cámara encontrados son:

print("# K intrinsics:")
print("K = ", np_print(K))

print()

print("# distortion coefficients:")
print("dist_coeffs = ", np_print(dist_coeffs))


print()

print("# image size:")
print("image_size = ", (w, h))


# Calibración con Tool de Calibración

La propuesta es utilizar la
[herramienta de calibración de cámaras](https://github.com/udesa-vision/i308-calib)

**NOTA: esta herramienta utiliza la GUI de ventanas del sistema operativo No funcionará en Google Colab**.


Necesitaremos un checkerboard físico con el que calibraremos. **Importante** asegurarse de que está pegado en una base rígida para evitar que se pandee.


## Instalación:

Se puede instalar via pip:

```bash

    pip install -qq git+https://github.com/udesa-vision/i308-calib.git

```

## Configurar herramienta

Ejecutar el comando que inicia la herramienta de calibración monocular:

```bash

    calib-tool

```

### Argumentos

- **video** str or int. 
Especifica el dispositivo de video, en linux podría ser algo del estilo `/dev/video<N>`

- **resolution** str (optional)
 la resolución solicitada en el formato "`<width>`x`<height>`" en pixels

- **checkerboard** str (optional) default=10x7
 distribución del checkerboard "`<width>`x`<height>`" en cuadrados

- **square-size** str (optional) default=24.2
 el tamaño de cuadrado en milímetros

- **config** str (optional)
un archivo .yaml de configuración de captura

- **data** str (optional) default=data
directorio donde se guardará el dataset de calibración, capturas y los resultados de calibración



### Ejemplos de comandos

Calibrar la cámara 0 con los parámetros por default:

```bash

    calib-tool --video 0

```


Calibrar con parámetros custom:

```bash

    calib-tool --video /dev/video3 --resolution 640x480 --checkerboard 9x6 --square-size 32.0 --data data_dir

```


Calibrar con archivo de configuración de captura:

```bash

    calib-tool --config cfg/capture.yaml

```


### Archivo de configuración de captura

Se proporcionan algunos archivos de configuración.

Para copiar los archivos de configuración al directorio de trabajo, ejecuta el comando:


```bash

 copy-configs

```

Esto debería crear la carpeta `cfg/` con algunos archivos de configuración.


Ejemplo de archivo de configuración (.yaml):

```yaml

    # video device, on linux might be /dev/video<N>
    video: 0

    # name: str (optional)
    #   a name to identify the device
    name: My Cute Camera

    # resolution: str (optional)
    #   requested resolution in the format "<width>x<height>" in pixels
    resolution: 640x480


```

Podemos editar el archivo o crear uno nuevo para configurar nuestro dispositivo.


### Sobreescribiendo parámetros de captura

En caso de utilizar archivo de configuración, algunos parametros como `video` y `resolution` se pueden sobreescribir, con los argumentos de la linea de comandos:

Por ejemplo, el siguiente comando va a utilizar el video 3, independientemente de lo que esté configurado en el archivo .yaml:

```bash

    calib-tool --video 3 --config cfg/capture.yaml 

```


##  Usando la herramienta

Al iniciar la herramienta vemos la captura de video.
Realizaremos distintos comandos con teclas de la computadora (la ventana debe tener el foco).  

<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/calb-tool/calib-tool-01.jpg?raw=true" width="75%"/>


---

Para activar / desactivar detección usar la tecla `d`.


<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/calb-tool/calib-tool-02.jpg?raw=true" width="75%"/>

---

Con la tecla `a` se pueden agregar elementos al dataset de calibración.
Las imágenes se guardarán en el directorio de `data/calib`.  

<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/calb-tool/calib-tool-03.jpg?raw=true" width="75%"/>


---

Vamos agregando imágenes al dataset de calibración hasta cubrir todas las regiones del sensor de la cámara.
Es importante variar lo más posible las posiciones y la orientación del checkerboard (ej, rotado 90º, más cerca, más lejos, en diagonal y con gran variación en el eje Z), esto dará mayor estabilidad al algoritmo de calibración.
En especial, los corners de la imagen, son los lugares de mayor distorsión y es conveniente tener ejemplos que cubran esas áreas.

Capturamos al menos 10 imagenes, idealmente más de 20.

<img src="https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_05/calb-tool/calib-tool-04.jpg?raw=true" width="75%"/>



---

Una vez que tenemos una buena cantidad de ejemplos presionamos la tecla `c` para obtener los parámetros de cámara:


```text

    calibrating...
    # Camera matrix : 
    
    K =  np.array([
            [  1455.282,         0.000,        946.688],
            [     0.000,      1456.869,        566.010],
            [     0.000,         0.000,          1.000]
    ])
    
    # Distortion Coefficients : 
    
    dist =  np.array([
            [  0.062529,     -0.221484,       0.000169,      -0.000469,       0.180346]
    ])


```


## Notas de uso
- presionando `q` terminamos el programa
- presionando `h` vemos el menú de ayuda con las teclas y sus correspondientes comandos.
- si algo sale catastróficamente mal simplemente borrar el directorio `data` y comenzar de nuevo.
- si la tool se cerró y queremos agregar nuevos ejemplos al dataset de calibración podemos cargar las imágenes del dataset con la tecla `l`
- Cuando el algoritmo no ve el tablero, tarda más que cuando lo ve. en caso de andar lento se puede apagar la deteccion con la letra `d` centrar el tablero y volver a activar la detección.
- presionando `s` podemos tomar capturas de objetos, que se guardarán en el directorio `data/captures`

# Distorsión

OpenCV modela distorsión radial y tangencial.

Los coeficientes de distorsión son $[k_1, k_2, p_1, p_2, k_3]$, en donde:

  - $k_1, k_2, k_3$ son los coeficientes de distorsión radial
  - $p_1, p_2$ son los coeficientes de distorsión tangencial


**Distorsión radial**
hace que las líneas rectas parezcan curvas, y aumenta cuanto más alejados están los puntos del centro de la imagen estemos.
Se modela como:

- $x_{distorted\_rad} = x . (1 + k_1 . r^2 + k_2 . r^4 + k_3 . r^6 )$
- $y_{distorted\_rad} = y . (1 + k_1 . r^2 + k_2 . r^4 + k_3 . r^6 )$

En donde:
- $ r = \sqrt{ x^2 + y ^ 2}$, es la distancia al centro 

**Distorsión tangencial** 
se produce porque la lente no está perfectamente alineada paralela al plano de la imagen. Por lo tanto, algunas áreas de la imagen pueden parecer más cercanas de lo esperado.
Se modela como:

- $x_{distorted\_tan} = x + [2  p_1  x  y + p_2  (r^2 + 2  x^2)] $
- $y_{distorted\_tan} = y + [p_1  (r^2 + 2  y^2) + 2  p_2  x y] $

**Corrección de distorsión** 


OpenCV primero normaliza las coordenadas para que queden relativas relativas al centro óptico.
OpenCV aplica ambas distorsiones en una única cuenta:

- $x_{distorted} = x . (1 + k_1 . r^2 + k_2 . r^4 + k_3 . r^6 ) + [2  p_1  x  y + p_2  (r^2 + 2  x^2)]  $
- $y_{distorted} = y . (1 + k_1 . r^2 + k_2 . r^4 + k_3 . r^6 ) + [p_1  (r^2 + 2  y^2) + 2  p_2  x y] $

Este modelo simula cómo se distorsiona un punto ideal y donde iría a parar en la imagen.
Pero para des-distorsionar necesitamos la operación inversa. Como invertir esta función es muy difícil, OpenCV utiliza métodos numéricos (Ej. Newton-Rhapson) para aproximar.


## Rectificando con OpenCV

Llamamos rectificar a la remoción de distorsión.
Ahora que tenemos el modelo de cámara con sus correspondientes coeficientes de distorsión, podemos eliminarla. 

OpenCV provee con dos métodos para des-distorsionar:

1. usando la función [`cv2.undistort`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga69f2545a8b62a6b0fc2ee060dc30559d), o bien
2. usando [`cv2.initUndistortRectifyMap`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga7dfb72c9cf9780a347fbe3d1c47e5d5a) y luego [`cv2.remap`](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gab75ef31ce5cdfb5c44b6da5f3b908ea4)


## Rectificando con cv2.undistort

Apliquemos cv2.undistort, para rectificar una imagen:


In [ ]:
# des-distorsionemos alguna imagen
image_index = 14

image = images[image_index]

undistorted = cv2.undistort(
    image, 
    K, 
    dist_coeffs
)

In [ ]:
axs = show_images([
    image, 
    undistorted
], [  
    "original",
    "undistorted",
])

## Optimal K - undistort ajustando bordes

Des-distorsionar puede traernos problemas en los bordes, recortando partes de la imagen, o dejarnos porciones en negro. Para eso OpenCV provee la función [`cv2.getOptimalNewCameraMatrix`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1), la cual nos devuelve una nueva matriz de cámara y una ROI, con el área óptima en la imagen desdistorsionada.

Esta función recibe el parámetro alpha, si lo seteamos en 0, va a cropear lo más posible la parte negra dejando la imagen centrada y sin bordes negros. Si lo seteamos en 1, va a mostrar toda la información de la imagen, incluso los bordes más des-distorsionados, pero puede introducir partes negras sin información, debido a la des-distorsión.



In [ ]:
def undistort(image, K, dist_coeffs, alpha):
    
    w, h = image.shape[1], image.shape[0]
    image_shape = (w, h)
    
    new_K, roi = cv2.getOptimalNewCameraMatrix(
        K, dist_coeffs, 
        image_shape, 
        alpha, 
        image_shape
    )

    undistorted = cv2.undistort(
        image, 
        K, 
        dist_coeffs,
        None,
        new_K
    )

    return undistorted, new_K, roi

In [ ]:
undistorted_0, K_0, roi_0 = undistort(image, K, dist_coeffs, alpha=0)
undistorted_1, K_1, roi_1 = undistort(image, K, dist_coeffs, alpha=1)

In [ ]:
show_images([
    image,
    undistorted,
    undistorted_0,
    undistorted_1
], [
    "original",
    "undistorted",
    "undistorted alpha=0",
    "undistorted alpha=1",
], grid=(2,2), figsize=(10, 6))

In [ ]:
from matplotlib import patches
ret = {}
fig, axes = show_images([
    undistorted_0,
    undistorted_1
], [
    "undistorted alpha=0",
    "undistorted alpha=1",
], show=False)

x, y, w, h = roi_0
axes[0].add_patch(patches.Rectangle(
    (x, y), w, h, linewidth=1, edgecolor='r', facecolor='none'
))
x, y, w, h = roi_1
axes[1].add_patch(patches.Rectangle(
    (x, y), w, h, linewidth=1, edgecolor='r', facecolor='none'
))
plt.show()
    

In [ ]:
import visualize_distortions

img_w, img_h = image_shape
visualize_distortions.visualize_2d_distortions(
    img_w, 
    img_h, 
    K, 
    dist_coeffs.reshape(-1),
    grid_step=40
    # grid_step=100
)

In [ ]:
visualize_distortions.visualize_3d_distortions(
    img_w, 
    img_h, 
    K, 
    dist_coeffs.reshape(-1),
    grid_step=int(img_h / 100),
    zlim=0.2
)

## Removiendo distorsión con cv2.initUndistortRectifyMap y cv2.remap

Invocar a la función undistort, requiere realizar un cómputo de resultados intermedios.

**Queremos des-distorsionar más rápido**

OpenCV provee la función cv2.initUndistortRectifyMap que devuelve estos resultados intermedios, que luego aplicaremos con la función cv2.remap.

En aplicaciones donde se necesita procesar múltiples imágenes, como en videos o secuencias de imágenes en aplicaciones de visión por computadora, calcular los mapas de transformación una sola vez con initUndistortRectifyMap y luego usar remap puede ser significativamente más eficiente.




In [ ]:
# Ejemplo de uso:

# 1. Obtengo la nueva matriz de cámara
alpha = 1  # Se puede usar alpha = 0, 1, o cualquier valor intermedio
new_K, roi = cv2.getOptimalNewCameraMatrix(K, dist_coeffs, image_shape, alpha, image_shape)

# 2: inicializamos los mapas de des-distorsion
map1, map2 = cv2.initUndistortRectifyMap(K, dist_coeffs, None, new_K, image_shape, cv2.CV_32FC1)

# 3: aplicamos remap para des-distorsionar la imagen
undistorted_image_faster = cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)

# 4: opcionalmente, podemos croppear la imagen usando el ROI devuelto en el paso 1.
#x, y, w, h = roi
#undistorted_image_cropped = undistorted_image[y:y+h, x:x+w]


In [ ]:
show_images([
    image,
    undistorted_1,
    undistorted_image_faster
], [
    "original",  "undistorted", "undistorted faster"
])

# Proyectando Puntos

OpenCV provee la función [`cv2.projectPoints`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga1019495a2c8d1743ed5cc23fa0daff8c) que dados los parámetros de cámara, nos permite proyectar puntos en el mundo en la imagen.




In [ ]:
# tomemos una imagen, con sus parámetros extrínsecos
image_index = 22

In [ ]:
image = images[image_index]
# corners = image_points[image_index]
checkerboard_world_points = world_points[image_index]
rv = rvecs[image_index]
tv = tvecs[image_index]

# Project 3D points to the 2D image plane
projected_points, _ = cv2.projectPoints(
    checkerboard_world_points, 
    rv, 
    tv, 
    K, 
    dist_coeffs
)
projected_points = projected_points.reshape(-1, 2).astype(int)

In [ ]:
_, ax = imshow(
    image,
    title="obs: son puntos proyectados. (no los detectados en la imagen)",
    figsize=(10, 6),
    show=False
)
ax.scatter(
    projected_points[:, 0],
    projected_points[:, 1],
    marker='+',
    color='r'
)

plt.show()

Podemos proyectar objetos virtuales como los ejes del marco de referencia del mundo
En este caso nuestra unidad métrica son casilleros del tablero.

In [ ]:
# axis_length = 200  # Length of the axis lines
#axis_len_x = 217.8 
#axis_len_y = 145.2
#axis_len_z = -100
axis_len_x = 9 * square_size_mm
axis_len_y = 6 * square_size_mm
axis_len_z = -6 * square_size_mm
axis_points = np.array([[0, 0, 0], 
                          [axis_len_x, 0, 0], 
                          [0, axis_len_y, 0], 
                          [0, 0, axis_len_z]], dtype=np.float32)


# Project 3D points to the 2D image plane
axis_points, _ = cv2.projectPoints(axis_points, rv, tv, K, dist_coeffs)

# Draw the axes on the image
axis_points = axis_points.reshape(-1, 2)
origin = tuple(axis_points[0].ravel())

# show_image = cv2.undistort(img, left_cam_matrix, left_dist_coeffs)
show_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

def draw_line(img, pt1, pt2, color, thickness=3):
    pt1 = (np.round(pt1[0]).astype(int), np.round(pt1[1]).astype(int))
    pt2 = (np.round(pt2[0]).astype(int), np.round(pt2[1]).astype(int))
    ret = cv2.line(img, pt1, pt2, color, thickness)
    return ret

draw_line(show_image, origin, axis_points[1], (0, 0, 255), 10)
draw_line(show_image, origin, axis_points[2], (0, 255, 0), 10)
draw_line(show_image, origin, axis_points[3], (255, 0, 0), 10)


_, ax = imshow(
    show_image,
    title="proyectamos los ejes del mundo (con -z)",
    figsize=(10, 6),
    show=False
)


plt.show()

In [ ]:
# podemos armar una funcion conveniente para graficar ejes con Matplotlib

def plot_line(ax, pt1, pt2, color, thickness=None, label=None):

    if thickness is None:
        thickness = 3
    
    ax.plot(
        [pt1[0], pt2[0]], 
        [pt1[1], pt2[1]],
        color=color,
        linewidth=thickness,
        label=label
    )
    
def plot_axis(ax, axis, thickness=None, labels=None):
    origin = axis[0]
    red = (1, 0, 0)
    green = (0, 1, 0)
    blue = (0, 0, 1)
    if labels is None:
        labels = ["x", "y", "z"]
    plot_line(ax, origin, axis_points[1], red, thickness, labels[0])
    plot_line(ax, origin, axis_points[2], green, thickness, labels[1])
    plot_line(ax, origin, axis_points[3], blue, thickness, labels[2])

    ax.legend()


fig, ax = imshow(image, figsize=(10, 6), show=False)

plot_axis(ax, axis_points, labels=["x", "y", "(-z)"])

plt.show()

## Manejando proyecciones con la distorsión


In [ ]:
# Tomemos una imagen de la calibración, con sus
# - corners detectados en la imagen,
# - correspondientes puntos en el mundo,
# - y parámetros extrínsecos de la calibración

In [ ]:
image_index = 22

image = images[image_index]
checkerboard_image_points = image_points[image_index]
checkerboard_world_points = world_points[image_index]
rv = rvecs[image_index]
tv = tvecs[image_index]

In [ ]:
img_pts = checkerboard_image_points.reshape(-1, 2)
_, ax = imshow(
    image, 
    title="estos son los corners detectados",
    figsize=(10, 6),
    show=False
)
ax.scatter(img_pts[:, 0], img_pts[:, 1], color='r', s=5, marker='+')

plt.show()

In [ ]:
# si proyectamos los puntos del checkerboard obtenemos:

checkerboard_projected_points, _ = cv2.projectPoints(
    checkerboard_world_points,
    rv, 
    tv, 
    K, 
    dist_coeffs
)

In [ ]:
proj_pts = checkerboard_projected_points.reshape(-1, 2)

_, ax =imshow(
    image, 
    "estos son puntos del mundo proyectados (usando K, dist, rvec y tvec)",
    figsize=(10, 6),
    show=False
)
ax.scatter(proj_pts[:, 0], proj_pts[:, 1], color='lime', s=5, marker='+')
plt.show()

In [ ]:
# Pero si quitamos distorsión en la imagen, 
# los puntos proyectados caen en posiciones incorrectas:

undistorted_image = cv2.undistort(
    image,
    K,
    dist_coeffs
)

In [ ]:
_, ax = imshow(
    undistorted_image,
    "imagen rectificada. los puntos proyectados no coinciden",
    figsize=(10, 6),
    show=False
)
ax.scatter(proj_pts[:, 0], proj_pts[:, 1], color='r', s=5)
plt.show()

In [ ]:
# Eso lo podemos arreglar usando la funcion cv2.undistortImagePoints,
# que des-distorsiona coordenadas de la imagen:
proj_pts_undistorted = cv2.undistortImagePoints(
    proj_pts,
    K,
    dist_coeffs
)


proj_pts_undistorted = proj_pts_undistorted.reshape(-1, 2)
_, ax = imshow(
    undistorted_image,
    title="imagen rectificada - puntos rectificados con cv2.undistortImagePoints()",
    figsize=(10, 6),
    show=False
)
ax.scatter(
    proj_pts_undistorted[:, 0], 
    proj_pts_undistorted[:, 1], 
    color='lime', 
    s=5
)

plt.show()

In [ ]:
# o bien, si partimos de puntos en el mundo,
# podemos remover la distorsión de la siguiente manera:

In [ ]:
# calculamos la nueva matriz de calibración, dado alpha en [0 .. 1]
alpha = 1.0
new_K, roi = cv2.getOptimalNewCameraMatrix(
    K, 
    dist_coeffs, 
    image_shape, 
    alpha,
    image_shape
)

In [ ]:
# rectificamos la imagen usando la nueva matriz de camara.
undistorted_image_optim = cv2.undistort(
    image,
    K,
    dist_coeffs,
    None,
    new_K
)

In [ ]:
# proyectamos los puntos. 
# OBS: usando la nueva matriz de cámara y SIN distorsión
optim_projected, _ = cv2.projectPoints(
    checkerboard_world_points, 
    rv, 
    tv, 
    new_K, 
    np.zeros_like(dist_coeffs)  # No distortion for undistorted image
)
optim_projected = optim_projected.reshape(-1, 2)

In [ ]:
_, ax = imshow(
    undistorted_image_optim,
    title="imagen rectificada, puntos del mundo proyectados con new_K, dist=0",
    figsize=(10, 6),
    show=False,
)
ax.scatter(optim_projected[:, 0], optim_projected[:, 1], color='lime', s=5)

plt.show()

# Pose de la cámara

Calcular la pose consiste en resolver la rotación y traslación que minimiza el error de reproyección a partir de correspondencias de puntos 3D-2D.

<img src="https://docs.opencv.org/4.x/pnp.jpg" />

## Perspectiva-por-N-Puntos (PnP)
PnP o Perspectiva-por-N-Puntos, es un método que estima la pose de la cámara en relación a puntos conocidos (ej. los puntos de un objeto), es decir, la traslación y rotación de la cámara con respecto al objeto.

OpenCV nos provee la función [`cv2.solvePnP( "correspondencias", "parametros de cámara", ...) -> pose`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga549c2075fac14829ff4a58bc931c033d) que implementa este método, permitiéndonos encontrar la posición y orientación de la cámara dado un conjunto de puntos 3D en el mundo y sus correspondientes proyecciones 2D en la imagen.

**solvePnP flags**
OpenCV implementa [diferentes algoritmos para estimar la pose](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#gga357634492a94efe8858d0ce1509da869a9f589872a7f7d687dc58294e01ea33a5).
Podemos usar los distintos métodos mediante el argumento `flags`, por default utiliza un algoritmo iterativo. El método que utilizaremos dependerá de las características de nuestro problema.




In [ ]:
# Ejemplo de PnP

image_index = 6

image = images[image_index]
checkerboard_image_points = image_points[image_index]
checkerboard_world_points = world_points[image_index]
rv = rvecs[image_index]
tv = tvecs[image_index]

In [ ]:
imshow(
    image,
    figsize=(10, 6)
)

In [ ]:
# solvePnP devuelve:
# - exito? True si pudo estimar y False si no.
# - rvec: la rotación de la cámara -> En formato Rodrigues
# - tvec: la traslación de la cámara

# flags=cv2.SOLVEPNP_IPPE
# flags=cv2.SOLVEPNP_EPNP

ret, rvec, tvec = cv2.solvePnP(
    checkerboard_world_points,
    checkerboard_image_points,
    K,
    dist_coeffs,
)

In [ ]:
ret

In [ ]:
rvec, tvec

## Rodrigues

El vector de Rodrigues es una forma compacta de representar rotaciones en 3D mediante un vector de tres elementos. Este vector representa un eje de rotación y un ángulo, lo que permite expresar rotaciones sin necesidad de usar una matriz de rotación completa (3x3).

La dirección del vector representa la dirección de rotación (eje), y la magnitud del vector representa el ángulo de rotación en radianes (alrededor del eje).

Dado un vector de Rodrigues, podemos obtener una matriz de rotación R, usando la función de OpenCV [`cv2.Rodrigues`](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac).

In [ ]:
R, _ = cv2.Rodrigues(
    rvec
)

In [ ]:
R

In [ ]:
(R @ R.T).round(6)

In [ ]:
# o bien pasar de una matriz de rotación, a vectores de rodrigues usando la misma función.
rvec, cv2.Rodrigues(R)[0]

## Pose en runtime ¿Cómo la usamos?

Muchos problemas relacionados con cámara tienen dos fases:
- una primera fase de calibración (~training del modelo) y
- tiempo de ejecución o "runtime".



**Prerrequisitos**
  - Ya realizamos la calibración y tenemos los parámetros de cámara `K` y `dist_coeffs`
  - En la imagen se puede detectar un objeto conocido del cual tenemos sus coordenadas 3d
    en el marco de referencia del mundo `obj_world_points`. 

**Runtime**
  1. Detectamos el objeto en la imagen, y obtenemos `obj_image_points` que son los puntos en 2d del objeto.
  2. Establecemos correspondencias entre los puntos $3d \leftrightarrow 2d$.
  3. Usando las correspondencias y los parámetros de cámara, computamos PnP para obtener la pose `R`, `t`

Si sabemos la pose en runtime podemos:

- transformar coordenadas de objetos en el marco de referencia del mundo al marco de referencia de la cámara
- transformar las coordenadas de la cámara al marco de referencia del mundo

entre otras cosas...

**A. del mundo al marco de referencia de la cámara**

4. Con la pose podemos armar la matriz extrínseca $M_{ext}$

$M_{ext} 
= \begin{bmatrix}
    R & t \\
    0 & 1 \\
\end{bmatrix}
$

5. Transformamos los puntos del mundo a la cámara $ \widetilde{X_c} = M_{ext} * \widetilde{X_w}  $

**B. de la cámara al marco de referencia del mundo**

4. Para transformar puntos del sistema de coordenadas de la cámara al sistema de coordenadas del objeto, necesitamos la transformación inversa de $M_{ext}$. Que se calcula como:

${M_{ext}} ^{-1} 
= \begin{bmatrix}
    R^T & -R^T . t \\
    0 & 1 \\
\end{bmatrix}
$

En donde:
- $R^T$ es la inversa de la matriz de rotación R (ya que R es una matriz de ortogonal).
- $-R^T . t$ es la traslación inversa.


Esta matriz toma un punto 3d en el marco de referencia de la cámara, expresado en coordenadas homogeneas, y lo lleva a un punto en 3d en el marco de referencia del mundo.

5. Transformamos los puntos la la cámara al mundo $ \widetilde{X_w} = {M_{ext}} ^{-1} . \widetilde{X_c} $


### Ejemplos:

In [ ]:
# Tomemos algunas capturas de ejemplo
pose_indices = [3, 16, 23, 24]
pose_images = [images[i] for i in pose_indices]
show_images(pose_images)

#### Cómputo de poses

In [ ]:
def compute_poses(
    images,
    camera_calibration,
    obj_world_points,
    detector,
):

    K, dist_coeffs = camera_calibration

    ret = []
    
    for image in images:
        
        found, obj_image_points = detector(image)
        if not found:
            raise Exception("object not detected")

        # compute la pose
        pose = cv2.solvePnP(
            obj_world_points,
            obj_image_points,
            K,
            dist_coeffs,
        )
        ret.append(pose)
    return ret


obj_world_points = checkerboard_world_points
camera_calibration = (K, dist_coeffs)
detector = lambda image: calib.detect_board(checkerboard, image)

# Computa las poses
poses = compute_poses(
    pose_images,
    camera_calibration,
    obj_world_points,
    detector
)

In [ ]:
poses

#### A. Del mundo a la cámara


In [ ]:
def to_homo(points):
    h_points = points.reshape(-1, 3)
    ones = np.ones( (len(h_points), 1) )
    h_points = np.concatenate(
        (
            h_points, 
            ones
        ), axis=1
    )
    return h_points
    

def world_to_camera(
    pose,
    obj_world_points_h
):

    ret, rvec, tvec = pose
    if not ret:
        raise Exception("cannot estimate pose")
    
    # computamos Rodrigues, para obtener la matriz de rotación
    R, _ = cv2.Rodrigues(rvec)
    
    # armamos la matriz extrínseca:
    # Mext T 4x4 que transforma puntos en coordenadas del mundo
    # en puntos en coordenadas de la cámara:
    # x_c = Mext * x_w 
    Mext = np.column_stack((R, tvec))
    Mext = np.vstack((Mext, [0, 0, 0, 1])) 
    
    # Transformamos puntos en coords del mundo a coords de la cámara
    obj_points_cam = (Mext @ obj_world_points_h.T).T
    obj_points_cam = obj_points_cam[:, :3] / obj_points_cam[:, 3].reshape(-1, 1)
    
    return obj_points_cam



obj_world_points_h = to_homo(obj_world_points)


obj_points_cam = [
    world_to_camera(pose, obj_world_points_h)
    for pose in poses
]



In [ ]:
import matplotlib.colors as mcolors
import numpy as np

def linear_cmap(rgb):
    
    # Normalize the RGB values to the range [0, 1]
    
    # Create a colormap that transitions from black (0, 0, 0) to the target color
    colors = [(0, 0, 0), rgb]  # Start with black, end with target color
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", colors)
    return cmap


M = 1.0
m = 0.0

red = (M, m, m)
green = (m, M, m)
blue = (m, m, M)
cyan = (m, M, M)
magenta = (M, m, M)
yellow = (M, M, m)


colors = [red, green, cyan, magenta]
cmaps = [
    linear_cmap(color)
    for color in colors
]

In [ ]:
def plot_line3d(ax, pt1, pt2, color, thickness=None, label=None, style=None):

    if thickness is None:
        thickness = 3

    if style is None:
        style = 'solid'
    
    plt.plot(
        [pt1[0], pt2[0]], 
        [pt1[1], pt2[1]],
        [pt1[2], pt2[2]],
        color=color,
        linewidth=thickness,
        label=label,
        linestyle=style
    )
    
def plot_axis3d(
    ax, 
    axis, 
    thickness=None,
    style=None,
    labels=None,
):

    origin = axis[0]
    red = (1, 0, 0)
    green = (0, 1, 0)
    blue = (0, 0, 1)

    if labels is None:
        labels = ["x", "y", "z"]
    elif len(labels) < 3:
        labels = [None] * 3
    plot_line3d(ax, origin, axis[1], red, thickness, labels[0], style)
    plot_line3d(ax, origin, axis[2], green, thickness, labels[1], style)
    plot_line3d(ax, origin, axis[3], blue, thickness, labels[2], style)


def build_axis3d(scale, homo=False, right_handed=True):

    axis = np.vstack(
        (
            np.zeros(3), # origen
            np.eye(3) * scale, # crea la identidad 3x3, y multiplica por la escala
        )
    )
    if not right_handed:
        axis[3, 2] = -axis[3, 2]

    if homo:
        axis = to_homo(axis)
        axis = axis.T
    
    return axis

def plot_objects_camera(
    obj_points_cam,
    axis=None,
):

        
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    for opc, col in zip(obj_points_cam, colors):
        
        ax.scatter(*opc.T, marker='o', color=col, label="puntos del objeto")

    if axis is not None:
        plot_axis3d(ax, axis)
    
    ax.view_init(elev=10, azim=-130)  # elev=90 para rotar 90 grados sobre el eje X
    
    #plt.axis('equal')
    ax.set_xlim([-500, 500])
    ax.set_ylim([-500, 500])
    ax.set_zlim([0, 1000])
    
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("coordenadas de los objetos en el marco de referencia de la cámara")
    plt.tight_layout()
    plt.show()


axis_len = 6 * square_size_mm
axis = build_axis3d(axis_len)

plot_objects_camera(
    obj_points_cam,
    axis=axis
)

In [ ]:
show_images(pose_images, colormaps=cmaps)

### B. De la cámara al mundo

In [ ]:
def camera_to_world(
    pose,
    obj_world_points_h
):

    ret, rvec, tvec = pose
    if not ret:
        raise Exception("cannot estimate pose")
    
    # computamos Rodrigues, para obtener la matriz de rotación
    R, _ = cv2.Rodrigues(rvec)
    
    # armamos la matriz extrínseca inversa:
    # Mext_inv T 4x4 que transforma puntos en coordenadas de la cámara
    # en puntos en coordenadas del mundo:
    # x_w = Mext_inv * x_c 

    R_inv = R.T  # La inversa de R es su transpuesta
    t_inv = -R_inv @ tvec
    Mext_inv = np.column_stack((R_inv, t_inv))
    Mext_inv = np.vstack((Mext_inv, [0, 0, 0, 1]))
    
    # Transformamos puntos:
    ret = (Mext_inv @ obj_world_points_h).T
    ret = ret[:, :3]
    
    return ret


def plot_objects_world(
    world_object,
    obj_points_world
):
        
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # grafico el objeto
    ax.scatter(*world_object.T, marker='.', color='black', label="puntos del objeto")
    
    for axis_cam_w, color in zip(obj_points_world, colors):
        # graficamos los ejes de cámara en el mundo
        pt = axis_cam_w[0]
        ax.scatter(*pt.T, marker='o', color=color, s=30, label="pose de cámara")
        
        plot_axis3d(ax, axis_cam_w, labels=[])
    
    
    # Dibujo los ejes del mundo
    axis_w = build_axis3d(500, right_handed=False)
    plot_axis3d(ax, axis_w, thickness=1, style="dashed", labels=["x world", "y world", "z world (left handed)"])
    
    ax.legend()

    ax.view_init(elev=10, azim=-130)  # elev=90 para rotar 90 grados sobre el eje X
    
    
    plt.axis('equal')
    plt.tight_layout()
    plt.show()


# El objeto del marco de camara a transformar será: 
# los ejes de coordenadas de la cámara
# Que definimos según:
axis_len = 6 * square_size_mm
axis_cam = build_axis3d(axis_len, homo=True)

# para cada pose obtenemos el objeto transformado:
obj_points_world = [
    camera_to_world(pose, axis_cam)
    for pose in poses
]

plot_objects_world(
    checkerboard_world_points,
    obj_points_world
)

In [ ]:
show_images(pose_images, colormaps=cmaps)